## Server-Side: FastAPI Application

In [ ]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
import pandas as pd
from typing import List, Dict

app = FastAPI()

class DataFrameInput(BaseModel):
    data: List[Dict]  # Expecting a list of dictionaries as input

@app.post("/process-dataframe")
async def process_dataframe(input_data: DataFrameInput):
    # Convert the JSON data into a Pandas DataFrame
    df = pd.DataFrame(input_data.data)
    
    # Modify the DataFrame (example: add a new column)
    df["new_column"] = df["column1"] * 2  # Assuming 'column1' exists in the input
    
    # Convert the modified DataFrame back to JSON
    response_data = df.to_dict(orient="records")
    return {"data": response_data}


## Client-Side: Sending and Receiving Data

In [ ]:
import requests
import pandas as pd

# Example DataFrame to send
df = pd.DataFrame({
    "column1": [1, 2, 3],
    "column2": ["A", "B", "C"]
})

# Convert DataFrame to JSON format
data_to_send = {"data": df.to_dict(orient="records")}

# Send the POST request
response = requests.post("http://127.0.0.1:8000/process-dataframe", json=data_to_send)

# Convert the response JSON back into a DataFrame
if response.status_code == 200:
    returned_data = response.json()["data"]
    result_df = pd.DataFrame(returned_data)
    print(result_df)
else:
    print("Error:", response.status_code, response.text)


## Running the Server


```
uvicorn your_script_name:app --reload
'''
